In [ ]:
from Models import Models
import Common
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split

%load_ext tensorboard

## Load Model

In [ ]:
model = Models.create_2DCNN_MLP()

In [ ]:
model = Models.create_3DCNN()

In [ ]:
model = Models.create_LSTM()

In [ ]:
model = Models.create_Transfer_LSTM((40, 128, 128, 3))

## Training

In [ ]:
dataset = Common.evaluate_dataset("hmdb51/")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    dataset.path, tf.keras.utils.to_categorical(LabelEncoder().fit_transform(dataset.category), 51)
)

In [ ]:
for i in range(200):
    model.fit(Common.create_batch(X_train, y_train, 16, (128, 128), True, 40, True))

In [ ]:
model.evaluate(Common.create_batch(X_test, y_test, 16, (128, 128), True, 40, True))

## New

In [ ]:
model = tf.keras.models.load_model("models/2DCNN_MLP_grayscale")

In [ ]:
train = Common.evaluate_dataset("dataset/train/", shuffle=True)
X_train = train.path
y_train = tf.keras.utils.to_categorical(train.category, 51)

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in range(100):
    model.fit(Common.create_batch(X_train, y_train, batch_size=16, grayscale=True, normalize=True))
# resize_shape=None, grayscale=True, downsampling_frames=None, normalize=True))
#callbacks=[tensorboard_callback] )

In [ ]:
test = Common.evaluate_dataset("dataset\\test\\")
X_test = test.path
y_test = tf.keras.utils.to_categorical(test.category, 51)
model.fit(Common.create_batch(X_test, y_test, batch_size=16, grayscale=True, normalize=True))

In [ ]:
model.save("models/" + model.name + "_grayscale")

In [ ]:
%tensorboard --logdir logs/fit --port 6006 --host localhost